In [ ]:
%pip install bitshuffle
%pip install Pillow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import blimpy as bl
#from ultralytics import YOLO
import time
import os
import random
import psutil
import scipy.ndimage
from PIL import Image
from PIL import ImageDraw
from pathlib import Path
import logging
from concurrent.futures import ProcessPoolExecutor, as_completed
import cv2
from scipy.ndimage import label as connected_components
from skimage.filters import sobel
from skimage.segmentation import slic
from skimage.util import img_as_float
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from skimage.feature import canny, blob_log
from skimage.transform import probabilistic_hough_line
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_objects
%matplotlib inline

In [ ]:
df = pd.read_csv('/home/cgchoza/galaxies/complete_cadences_catalog.csv')
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
1,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
2,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
3,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
4,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
...,...,...,...,...,...,...,...,...
36553,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36554,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36555,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
36556,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [ ]:
df['.h5 path'] = df['.h5 path'].str.replace('0000.h5', '0002.h5', regex=False)
df = df.drop_duplicates(subset='.h5 path', keep='first').reset_index(drop=True)
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
1,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
2,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
3,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
4,DDO210,AGBT18A_999_103,L,24777,2251,/datag/pipeline/AGBT18A_999_103/collate/splice...,/home/obs/turboseti/AGBT18A_999_103/collate/sp...,2018-07-07 08:49:26
...,...,...,...,...,...,...,...,...
30309,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30310,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30311,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
30312,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [ ]:
df = df[~df['.h5 path'].str.contains('spliced')].reset_index(drop=True)
df

,Target,Session,Band,Cadence ID,Frequency,.h5 path,.dat path,Time
0,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
1,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
2,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
3,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
4,AND_XIV,AGBT18B_999_07,S,30225,3151,/datag/pipeline/AGBT18B_999_07/blc40_blp00/blc...,/home/obs/turboseti/AGBT18B_999_07/blc40_blp00...,2018-08-18 08:41:37
...,...,...,...,...,...,...,...,...
29341,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29342,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29343,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36
29344,NGC3226,AGBT22B_999_25,L,411390,1126,/datag/pipeline/AGBT22B_999_25/blc16_blp06/blc...,/home/obs/turboseti/AGBT22B_999_25/blc16_blp06...,2022-11-19 06:13:36


In [ ]:
df = df.drop(index = 17546)
df = df.sample(n=1000, random_state=42).reset_index(drop=True)

In [ ]:
fb = bl.Waterfall(df['.h5 path'].iloc[-1], load_data=True)
fb.info()


--- File Info ---
DIMENSION_LABELS : [b'time' b'feed_id' b'frequency']
        az_start :                              0.0
       data_type :                                1
            fch1 :               10501.46484375 MHz
            foff :         -0.00286102294921875 MHz
           ibeam :                               -1
      machine_id :                               20
          nbeams :                                1
           nbits :                               32
          nchans :                            65536
            nifs :                                1
     rawdatafile : guppi_59411_54386_HIP30264_0094.0000.raw
     source_name :                         HIP30264
         src_dej :                     -8:26:53.228
         src_raj :                      6:21:58.451
    telescope_id :                                6
           tsamp :                1.073741823999999
   tstart (ISOT) :          2021-07-16T15:06:26.000
    tstart (MJD) :                59

In [ ]:
def merge_boxes(boxes, iou_thresh=0.1, proximity_thresh=0.05):
    """
    Merge boxes if their centers are close or they have significant IoU overlap.
    boxes: list of (cx, cy, w, h, area)
    returns: merged list of boxes
    """
    import numpy as np

    def iou(b1, b2):
        x1_min = b1[0] - b1[2] / 2
        x1_max = b1[0] + b1[2] / 2
        y1_min = b1[1] - b1[3] / 2
        y1_max = b1[1] + b1[3] / 2

        x2_min = b2[0] - b2[2] / 2
        x2_max = b2[0] + b2[2] / 2
        y2_min = b2[1] - b2[3] / 2
        y2_max = b2[1] + b2[3] / 2

        inter_xmin = max(x1_min, x2_min)
        inter_ymin = max(y1_min, y2_min)
        inter_xmax = min(x1_max, x2_max)
        inter_ymax = min(y1_max, y2_max)

        inter_area = max(0, inter_xmax - inter_xmin) * max(0, inter_ymax - inter_ymin)
        area1 = b1[2] * b1[3]
        area2 = b2[2] * b2[3]
        union = area1 + area2 - inter_area

        return inter_area / union if union > 0 else 0

    def distance(b1, b2):
        return np.hypot(b1[0] - b2[0], b1[1] - b2[1])

    merged = []
    used = [False] * len(boxes)

    for i, b1 in enumerate(boxes):
        if used[i]:
            continue
        group = [b1]
        used[i] = True
        for j, b2 in enumerate(boxes):
            if used[j]:
                continue
            if iou(b1, b2) > iou_thresh or distance(b1, b2) < proximity_thresh:
                group.append(b2)
                used[j] = True
        if len(group) == 1:
            merged.append(group[0])
        else:
            # merge into one big box (average center, min enclosing width/height)
            xs = [b[0] for b in group]
            ys = [b[1] for b in group]
            ws = [b[2] for b in group]
            hs = [b[3] for b in group]
            xmins = [x - w/2 for x, w in zip(xs, ws)]
            xmaxs = [x + w/2 for x, w in zip(xs, ws)]
            ymins = [y - h/2 for y, h in zip(ys, hs)]
            ymaxs = [y + h/2 for y, h in zip(ys, hs)]
            new_x = (min(xmins) + max(xmaxs)) / 2
            new_y = (min(ymins) + max(ymaxs)) / 2
            new_w = max(xmaxs) - min(xmins)
            new_h = max(ymaxs) - min(ymins)
            merged.append((new_x, new_y, new_w, new_h, new_w * new_h))
    return merged


In [ ]:
# def edge_detection(gray, factor=6, dilation=(3, 3)):
#         # STEP 1: apply power threshold to suppress background
#         median = np.median(gray)
#         mad = np.median(np.abs(gray - median))
#         power_mask = gray > (median + factor * mad)

#         # STEP 2: edge detection on filtered signal only
#         grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
#         grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
#         magnitude = np.sqrt(grad_x**2 + grad_y**2)
#         magnitude *= power_mask  # mask out noise
#         magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

#         # STEP 3: binary threshold + dilation
#         binary = magnitude > 50  # you can tune this
#         binary = scipy.ndimage.binary_dilation(binary, structure=np.ones(dilation))
#         labeled, n_objs = scipy.ndimage.label(binary)
#         slices = scipy.ndimage.find_objects(labeled)
#         return slices

# EDGE_MODEL_PATH = '/home/jliang/gbt-rfi/model.yml.gz'
# EDGE_DETECTOR = cv2.ximgproc.createStructuredEdgeDetection(EDGE_MODEL_PATH)

# def detect_edges(spectrogram):
#     img = (spectrogram - spectrogram.min()) / (spectrogram.ptp() + 1e-6)
#     img_3ch = cv2.merge([img.astype(np.float32)] * 3)
#     return EDGE_DETECTOR.detectEdges(img_3ch)

In [ ]:
# def extract_one_sample(h5_path, ch):
#     try:
#         EDGE_DETECTOR = cv2.ximgproc.createStructuredEdgeDetection(edge_model_path)
#         fb = bl.Waterfall(h5_path, load_data=True)
#         data = 10 * np.log10(fb.data.squeeze())

#         nfpc = fb.header.get("nfpc", 1024)
#         f0, f1 = ch * nfpc, (ch + 1) * nfpc
#         block = data[:, f0:f1]

#         n_cols = block.shape[1]
#         low, high = int(0.1 * n_cols), int(0.9 * n_cols)
#         block_middle80 = block[:, low:high]

#         vert_means = block_middle80.mean(axis=0)
#         center = np.argmax(vert_means)
#         left_col, right_col = center - 1, center + 1
#         if left_col >= 0 and right_col < block_middle80.shape[1]:
#             block_middle80[:, center] = (block_middle80[:, left_col] + block_middle80[:, right_col]) / 2
#         elif left_col >= 0:
#             block_middle80[:, center] = block_middle80[:, left_col]
#         elif right_col < block_middle80.shape[1]:
#             block_middle80[:, center] = block_middle80[:, right_col]

#         img_norm = (block_middle80 - block_middle80.min()) / (block_middle80.ptp() + 1e-6)
#         img_rgb = cv2.merge([img_norm.astype(np.float32)] * 3)

#         sf_edges = EDGE_DETECTOR.detectEdges(img_rgb)

#         gray = (255 * img_norm).astype(np.uint8)
#         grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
#         grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
#         magnitude = np.sqrt(grad_x**2 + grad_y**2)
#         magnitude = cv2.normalize(magnitude, None, 0, 1, cv2.NORM_MINMAX)

#         sf_mean = np.mean(sf_edges)
#         ued_mean = np.mean(magnitude)

#         if sf_mean + ued_mean < 0.005:  # threshold to skip empty or background-only blocks
#             return None  # skip empty or background-only blocks


#         return (img_rgb, sf_edges, magnitude)

#     except Exception as e:
#         print(f"Error processing {h5_path} ch {ch}: {e}")
#         return None

# def extract_samples_parallel(df: pd.DataFrame, max_samples=500, num_workers=8):
#     from random import shuffle

#     all_tasks = []
#     for _, row in df.iterrows():
#         h5 = row[".h5 path"]
#         fb = bl.Waterfall(h5, load_data=False)
#         nfreq = fb.header.get("nchans")
#         nfpc = fb.header.get("nfpc", 1024)
#         for ch in range(nfreq // nfpc):
#             all_tasks.append((h5, ch))

#     shuffle(all_tasks)
#     selected_tasks = all_tasks[:max_samples]

#     train_images, train_sf_edges, train_ued_edges = [], [], []

#     with ProcessPoolExecutor(max_workers=num_workers) as executor:
#         futures = [executor.submit(extract_one_sample, h5, ch) for h5, ch in selected_tasks]
#         for future in as_completed(futures):
#             result = future.result()
#             if result:
#                 img_rgb, sf_edge, ued_edge = result
#                 train_images.append(img_rgb)
#                 train_sf_edges.append(sf_edge)
#                 train_ued_edges.append(ued_edge)

#     return train_images, train_sf_edges, train_ued_edges


In [ ]:
# 4) SET UP LOGGING
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)


def build_tasks(df):
    """
    Build a flat list of (h5_path, channel_idx) from your DataFrame.
    """
    tasks = []
    for _, row in df.iterrows():
        h5 = row[".h5 path"]
        fb = bl.Waterfall(h5, load_data=False)
        nfreq = fb.header["nchans"]
        nfpc = fb.header.get("nfpc", 1024)
        n_coarse = nfreq // nfpc

        for ch in range(n_coarse):
            tasks.append((h5, ch))
    return tasks


def split_tasks(tasks, train_frac=0.8, seed=42):
    """
    Shuffle & split the flat task list into train vs. val sets.
    Returns two sets of (h5_path, channel_idx).
    """
    random.seed(seed)
    shuffled = tasks.copy()
    random.shuffle(shuffled)
    cut = int(train_frac * len(shuffled))
    train = set(shuffled[:cut])
    val = set(shuffled[cut:])
    return train, val


def generate_yolo_boxes(edge_map, threshold=0.1):
    """
    Turn an edge map into YOLO‐style boxes + area.
    """
    binary = (edge_map > threshold).astype(np.uint8)
    # label connected components
    labeled = label(binary)
    boxes = []
    for region in regionprops(labeled):
        if region.area < 10:
            continue
        y0, x0, y1, x1 = region.bbox
        cx = (x0 + x1) / 2 / edge_map.shape[1]
        cy = (y0 + y1) / 2 / edge_map.shape[0]
        w = (x1 - x0) / edge_map.shape[1]
        h = (y1 - y0) / edge_map.shape[0]
        boxes.append((cx, cy, w, h, w * h))
    return boxes


def is_inside(inner, outer):
    """
    Return True if box `inner` lies (mostly) inside box `outer`.
    Boxes are (cx, cy, w, h, area).
    """
    ix, iy, iw, ih, _ = inner
    ox, oy, ow, oh, _ = outer
    return (
        abs(ix - ox) < 0.5 * (ow - iw)
        and abs(iy - oy) < 0.5 * (oh - ih)
    )


def process_file(job):
    """
    job is a tuple of:
      (h5_path, channels, global_indices,
       base_dir, factor, dilation_size,
       class_id, train_set, val_set,
       pad_width, model, scaler)
    """
    (
        h5_path,
        channels,
        global_indices,
        base_dir,
        factor,
        dilation_size,
        class_id,
        train_set,
        val_set,
        pad_width
    ) = job

    # load once per file
    fb = bl.Waterfall(h5_path, load_data=True)
    data = 10 * np.log10(fb.data.squeeze())  # (ntime, nfreq)

    for ch_idx, gidx in zip(channels, global_indices):
        # split train vs val
        subset = "train" if (h5_path, ch_idx) in train_set else "val"

        # setup directories
        img_dir = Path(base_dir) / subset / "images"
        lbl_dir = Path(base_dir) / subset / "labels"
        vis_dir = Path(base_dir) / "visualization" / subset
        for d in (img_dir, lbl_dir, vis_dir):
            d.mkdir(parents=True, exist_ok=True)

        # extract the coarse channel block
        cw = fb.header.get("nfpc", 1024)
        f0, f1 = ch_idx * cw, (ch_idx + 1) * cw
        block = data[:, f0:f1]

        # crop out the roll-off edges
        n_cols = block.shape[1]
        lo, hi = int(0.1 * n_cols), int(0.9 * n_cols)
        block = block[:, lo:hi]

        # remove a vertical artifact
        means = block.mean(axis=0)
        center = np.argmax(means)
        L, R = center - 1, center + 1
        if 0 <= L < n_cols and 0 <= R < n_cols:
            block[:, center] = 0.5 * (block[:, L] + block[:, R])

        # normalize for processing
        norm = (block - block.min()) / (block.ptp() + 1e-6)

        # —— DETECTION STEPS —— #
        edges = canny(norm, sigma=2)
        spec_norm = (norm - norm.min()) / (norm.ptp() + 1e-6)
        th = threshold_otsu(norm)
        mask = remove_small_objects(norm > th, min_size=100)

        # run detectors
        lines   = probabilistic_hough_line(edges, threshold=10, line_length=30, line_gap=3)
        blobs   = blob_log(spec_norm, min_sigma=2, max_sigma=20, num_sigma=25, threshold=0.03, overlap=0.5)
        regions = [r for r in regionprops(label(mask)) if r.area > 40]

        # generate YOLO boxes
        all_boxes = generate_yolo_boxes(edges, threshold=0.1)
        all_boxes.sort(key=lambda b: b[4], reverse=True)
        final_boxes = []
        for box in all_boxes:
            if not any(is_inside(box, kept) for kept in final_boxes):
                final_boxes.append(box)

        # —— SAVE IMAGE & LABELS —— #
        # image filename
        fn = f"img_{gidx:0{pad_width}d}.png"
        img_path = img_dir / fn

        # save greyscale block as RGB PNG
        arr8 = (255 * norm).astype(np.uint8)
        img = Image.fromarray(np.stack([arr8]*3, axis=-1))
        img.save(img_path)

        # save YOLO label file
        txt_path = lbl_dir / fn.replace(".png", ".txt")
        if final_boxes:
            with open(txt_path, "w") as fout:
                for cx, cy, w, h, _ in final_boxes:
                    if w * h < 0.95 and min(w, h) * block.shape[1] > 3:
                        fout.write(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")
        elif txt_path.exists():
            txt_path.unlink()

        # overlay and save visualization
        vis = img.convert("RGB")
        draw = ImageDraw.Draw(vis)
        for cx, cy, w, h, _ in final_boxes:
            x0 = int((cx - w/2) * vis.width)
            y0 = int((cy - h/2) * vis.height)
            x1 = int((cx + w/2) * vis.width)
            y1 = int((cy + h/2) * vis.height)
            draw.rectangle([x0, y0, x1, y1], outline="red", width=2)
        vis.save(vis_dir / fn)


if __name__ == "__main__":
    # your settings
    BASE_DIR    = "/datax/scratch/jliang/dataset_hough"
    FACTOR      = 6
    DILATION    = (30, 30)
    CLASS_ID    = 0
    TRAIN_FRAC  = 0.8
    SEED        = 42
    NUM_WORKERS = 8

    # build & split
    tasks    = build_tasks(df)
    train_set, val_set = split_tasks(tasks, TRAIN_FRAC, SEED)

    # zero-pad width
    pad_width = len(str(len(tasks)))

    # group by file so we only open each .h5 once
    jobs = {}
    for gidx, (h5, ch) in enumerate(tasks):
        jobs.setdefault(h5, {"chs": [], "gidxs": []})
        jobs[h5]["chs"].append(ch)
        jobs[h5]["gidxs"].append(gidx)

    # prepare job tuples (we’ll append model & scaler below)
    job_list = []
    for h5, info in jobs.items():
        job_list.append((
            h5,
            info["chs"],
            info["gidxs"],
            BASE_DIR,
            FACTOR,
            DILATION,
            CLASS_ID,
            train_set,
            val_set,
            pad_width
        ))


    
    with ProcessPoolExecutor(max_workers=NUM_WORKERS) as exe:
        futures = [exe.submit(process_file, job) for job in job_list]
        for _ in as_completed(futures):
            pass

    logging.info("All done.")

In [ ]:
# # 4) SET UP LOGGING
# logging.basicConfig(
#     level=logging.INFO,
#     format="%(asctime)s [%(levelname)s] %(message)s",
#     datefmt="%Y-%m-%d %H:%M:%S",
# )

# def build_tasks(df):
#     """
#     Build a flat list of (h5_path, channel_idx) from your DataFrame.
#     Uses df['nchans'] if present, otherwise reads header to find nfreq.
#     """
#     tasks = []
#     for _, row in df.iterrows():
#         h5 = row[".h5 path"]
#         fb = bl.Waterfall(h5, load_data=False)
#         nfreq = fb.header.get("nchans")
#         nfpc = fb.header.get("nfpc", 1024)  # default to 1024 if not set
#         n_coarse = nfreq // nfpc

#         for ch in range(n_coarse):
#             tasks.append((h5, ch))
#     return tasks

# def split_tasks(tasks, train_frac=0.8, seed=42):
#     """
#     Shuffle & split the flat task list into train vs. val sets.
#     Returns two sets of (h5_path, channel_idx).
#     """
#     random.seed(seed)
#     shuffled = tasks.copy()
#     random.shuffle(shuffled)
#     cut = int(train_frac * len(shuffled))
#     train = set(shuffled[:cut])
#     val   = set(shuffled[cut:])
#     return train, val

# def process_file(job):
#     """
#     job is a tuple:
#       (h5_path, [ch_idx,...], [global_idx,...],
#        base_dir, coarse_channel_width, factor, dilation_size,
#        class_id, train_set, val_set)
#     """
#     (h5_path,
#      channels,
#      global_indices,
#      base_dir,
#      factor,
#      dilation,
#      class_id,
#      train_set,
#      val_set,
#      pad_width,
#      model,
#      scalar) = job

#     def generate_yolo_boxes(edge_map, threshold=0.1):
#         binary = (edge_map > threshold).astype(np.uint8)
#         labeled, n = connected_components(binary)
#         boxes = []
#         for i in range(1, n + 1):
#             coords = np.argwhere(labeled == i)
#             if coords.shape[0] < 10:
#                 continue
#             y0, x0 = coords.min(axis=0)
#             y1, x1 = coords.max(axis=0)
#             cx = (x0 + x1) / 2 / edge_map.shape[1]
#             cy = (y0 + y1) / 2 / edge_map.shape[0]
#             w = (x1 - x0) / edge_map.shape[1]
#             h = (y1 - y0) / edge_map.shape[0]
#             boxes.append((cx, cy, w, h, w * h))
#         return boxes

#     # 1) load the waterfall once
#     fb   = bl.Waterfall(h5_path, load_data=True)
#     data = 10 * np.log10(fb.data.squeeze())   # shape (ntime, nfreq)
#     nt, nf = data.shape

#     #  # 2) threshold + dilation
#     # M, S = np.median(data), np.std(data)
#     # binary = data > (M + factor * S)
#     # binary = scipy.ndimage.maximum_filter(binary, size=dilation)

#     # 3) process each channel
#     for ch_idx, gidx in zip(channels, global_indices):
#         cw = fb.header.get("nfpc", 1024)  # default to 1024 if not set
#         f0, f1 = ch_idx*cw, (ch_idx+1)*cw
#         block  = data[:, f0:f1]

#         # decide train vs val
#         subset = "train" if (h5_path, ch_idx) in train_set else "val"

#         # make sure all dirs exist
#         img_dir = Path(base_dir)/subset/"images"
#         lbl_dir = Path(base_dir)/subset/"labels"
#         vis_dir = Path(base_dir)/"visualization"/subset
#         for d in (img_dir, lbl_dir, vis_dir):
#             d.mkdir(parents=True, exist_ok=True)

#         # Get rid of rolloff
#         n_cols = block.shape[1]
#         low = int(0.1 * n_cols)
#         high = int(0.9 * n_cols)
#         block_middle80 = block[:, low:high]

#         # Remove vertical line artifact
#         rows, cols = block_middle80.shape
#         vert_means = block_middle80.mean(axis=0)
#         center = np.argmax(vert_means)
#         left_col = center - 1
#         right_col = center + 1
#         if left_col >= 0 and right_col < cols:
#             block_middle80[:, center] = (block_middle80[:, left_col] + block_middle80[:, right_col]) / 2
#         elif left_col >= 0:
#             block_middle80[:, center] = block_middle80[:, left_col]
#         elif right_col < cols:
#             block_middle80[:, center] = block_middle80[:, right_col]

#         # ====== NOW threshold and dilate on the CLEANED block ======
#         #import scipy.ndimage

#         # Normalize for visualization + processing
#         arr8 = (255 * (block_middle80 - block_middle80.min()) / block_middle80.ptp()).astype(np.uint8)
#         gray = arr8
#         h, w = gray.shape

#         # Normalize the image block
#         img_norm = (block_middle80 - block_middle80.min()) / (block_middle80.ptp() + 1e-6)

#         # Common precomputations
#         # 1) Edge map for Hough & (if desired) for Generalized Hough
#         edges = canny(img_norm, sigma=2)

#         # 2) Normalized spec for LoG
#         spec_norm = (img_norm - img_norm.min()) / img_norm.ptp()

#         # 3) Mask for connected components
#         thresh = threshold_otsu(img_norm)
#         mask = img_norm > thresh
#         mask = remove_small_objects(mask, min_size=100)

#         # Define each detector as its own function
#         def detect_lines(edges):
#             return probabilistic_hough_line(
#                 edges,
#                 threshold=10,
#                 line_length=50,
#                 line_gap=3
#             )

#         def detect_blobs(spec_norm):
#             # returns array of (y, x, sigma)
#             return blob_log(
#                 spec_norm,
#                 min_sigma=2,
#                 max_sigma=20,
#                 num_sigma=10,
#                 threshold=0.03
#             )

#         def detect_regions(mask):
#             labels = label(mask)
#             # filter out tiny regions
#             return [r for r in regionprops(labels) if r.area > 200]

#         # Run all three in parallel
#         with ProcessPoolExecutor() as exe:
#             future_lines   = exe.submit(detect_lines, edges)
#             future_blobs   = exe.submit(detect_blobs, spec_norm)
#             future_regions = exe.submit(detect_regions, mask)

#             lines   = future_lines.result()
#             blobs   = future_blobs.result()
#             regions = future_regions.result()

#         # Visualization (serial)
#         fig, ax = plt.subplots(figsize=(10, 6))
#         ax.imshow(img_norm, cmap='gray', origin='lower')

#         # overlay Hough lines in red
#         for (x0, y0), (x1, y1) in lines:
#             ax.plot((x0, x1), (y0, y1), '-r', linewidth=1)

#         # overlay LoG blobs in yellow circles
#         for y, x, sigma in blobs:
#             circ = plt.Circle((x, y), sigma * np.sqrt(2),
#                               edgecolor='yellow', facecolor='none', linewidth=1)
#             ax.add_patch(circ)

#         # overlay connected‐component boxes in lime
#         for region in regions:
#             minr, minc, maxr, maxc = region.bbox
#             rect = plt.Rectangle((minc, minr),
#                                 maxc - minc,
#                                 maxr - minr,
#                                 edgecolor='lime',
#                                 facecolor='none',
#                                 linewidth=1)
#             ax.add_patch(rect)

#         ax.set_axis_off()
#         plt.tight_layout()
#         plt.show()

#         # Calculate frequency range of this coarse channel
#         f_start = fb.header['fch1'] + f0 * fb.header['foff']
#         f_stop  = fb.header['fch1'] + (f1 - 1) * fb.header['foff']

#         # Prepare and save PNG
#         arr8 = (255 * (block_middle80 - block_middle80.min()) / block_middle80.ptp()).astype(np.uint8)
#         rgb = np.stack([arr8] * 3, axis=-1)
#         img = Image.fromarray(rgb)
#         fn = f"img_{gidx:0{pad_width}d}_f_{f_start:.4f}_{f_stop:.4f}.png"
#         img_path = img_dir / fn
#         img.save(img_path)

#                 # Save YOLO boxes
#         txt_path = lbl_dir / fn.replace(".png", ".txt")
#         min_size = 3
#         full_image_threshold = 0.95  # percentage of area to reject

#         # Apply greedy suppression: remove boxes fully inside larger ones
#         def is_inside(inner, outer):
#             ix, iy, iw, ih = inner[:4]
#             ox, oy, ow, oh = outer[:4]
#             return (
#                 abs(ix - ox) < 0.5 * (ow - iw) and
#                 abs(iy - oy) < 0.5 * (oh - ih)
#             )

#         edge_map = detect_edges(block_middle80)
#         all_boxes.extend(generate_yolo_boxes(edge_map, threshold=0.1))
#         all_boxes.sort(key=lambda b: b[4], reverse=True)  # sort by area
#         merged_boxes = merge_boxes(all_boxes, iou_thresh=0.1, proximity_thresh=0.05)
#         final_boxes = []
#         for box in merged_boxes:
#             if not any(is_inside(box, keep) for keep in final_boxes):
#                 final_boxes.append(box)


#         # Write to YOLO format
#         with open(txt_path, "w") as ftxt:
#             for x_c, y_c, w_n, h_n, _ in final_boxes:
#                 if w_n * h_n > full_image_threshold:
#                     continue
#                 if w_n * w < min_size or h_n * h < min_size:
#                     continue
#                 ftxt.write(f"{class_id} {x_c:.6f} {y_c:.6f} {w_n:.6f} {h_n:.6f}\n")

#         if not final_boxes:
#             if txt_path.exists():
#                 txt_path.unlink()
#             with open(lbl_dir / 'empty_labels.csv', 'a') as f:
#                 f.write(f"{fn},{h5_path},{ch_idx}\n")




#         # —————————————————————————————
#         # 4) Draw boxes on the saved PNG
#         # —————————————————————————————
#         vis_img = Image.open(img_path).convert("RGB")
#         draw    = ImageDraw.Draw(vis_img)
#         w_img, h_img = vis_img.size


#         if txt_path.exists():
#             with open(txt_path) as ftxt:
#                 for line in ftxt:
#                     cls, x_c, y_c, bw, bh = map(float, line.split())
#                     xc = x_c * w_img
#                     yc = y_c * h_img
#                     bw_pix = bw * w_img
#                     bh_pix = bh * h_img
#                     x0 = int(xc - bw_pix/2)
#                     y0 = int(yc - bh_pix/2)
#                     x1 = int(xc + bw_pix/2)
#                     y1 = int(yc + bh_pix/2)
#                     draw.rectangle([x0, y0, x1, y1], outline="red", width=2)

#         vis_img.save(vis_dir/fn)


# if __name__ == "__main__":
#     # === your settings ===
#     BASE_DIR             = "/datax/scratch/jliang/dataset_moe"
#     FACTOR              = 6
#     DILATION = (30, 30)
#     CLASS_ID             = 0
#     TRAIN_FRAC           = 0.8
#     SEED                 = 42
#     NUM_WORKERS          = 8

#     # --- assume you already have a DataFrame `df` with at least 'h5_path' and optionally 'nchans' ---
#     # df = pd.read_csv(...)  # or however you built it

#     # 1) Build & split tasks
#     tasks = build_tasks(df)
#     train_set, val_set = split_tasks(tasks, TRAIN_FRAC, SEED)

#     # 2) compute zero-pad width from total images
#     pad_width = len(str(len(tasks) - 1))

#     # 3) regroup tasks by file to load each .h5 only once
#     jobs = {}
#     for gidx, (h5, ch) in enumerate(tasks):
#         jobs.setdefault(h5, {"chs": [], "gidxs": []})
#         jobs[h5]["chs"].append(ch)
#         jobs[h5]["gidxs"].append(gidx)

#     # 4) prepare job‐tuples
#     job_list = []
#     for h5, info in jobs.items():
#         job_list.append((
#             h5,
#             info["chs"],
#             info["gidxs"],
#             BASE_DIR,
#             FACTOR,
#             DILATION,
#             CLASS_ID,
#             train_set,
#             val_set,
#             pad_width
#         ))

#     # 5) run in parallel, one process per file
#     # with ProcessPoolExecutor(max_workers=NUM_WORKERS) as exe:
#     #     for _ in as_completed(exe.map(process_file, job_list)):
#     #         pass
#     train_images, train_sf_edges, train_ued_edges = extract_samples_parallel(df, max_samples=500, num_workers=8)
#     # Load or train your MoE model here
#     model, scaler = train_gating_model(train_images, train_sf_edges, train_ued_edges)

#     with ProcessPoolExecutor(max_workers=NUM_WORKERS) as exe:
#         futures = [exe.submit(process_file, job + (model, scaler)) for job in job_list]
#         for future in as_completed(futures):
#             result = future.result()

#     logging.info("All done.")
